In [32]:
from aeon.datasets import load_classification
X, y, meta_data = load_classification("Epilepsy2")
print(" Shape of X = ", X.shape)
print(" Meta data = ", meta_data)

 Shape of X =  (11500, 1, 178)
 Meta data =  {'problemname': 'cardiacarrhythmia_combo.ts', 'timestamps': False, 'missing': False, 'univariate': True, 'equallength': True, 'classlabel': True, 'targetlabel': False, 'class_values': ['0', '1']}


In [33]:
X.shape

(11500, 1, 178)

In [34]:
y.shape

(11500,)

In [36]:
meta_data

{'problemname': 'cardiacarrhythmia_combo.ts',
 'timestamps': False,
 'missing': False,
 'univariate': True,
 'equallength': True,
 'classlabel': True,
 'targetlabel': False,
 'class_values': ['0', '1']}

In [37]:
import pandas as pd
df = pd.DataFrame(data=X[:, 0, :])
df['label'] = y
df.head()

,0,1,2,3,4,5,6,7,8,9,...,169,170,171,172,173,174,175,176,177,label
0,0.341655,0.349197,0.374292,0.412786,0.427319,0.388749,0.377159,0.372810,0.385130,0.399336,...,0.383370,0.448307,0.449510,0.452042,0.448180,0.428945,0.412748,0.411765,0.449786,0
1,0.348107,0.388060,0.441110,0.466879,0.452705,0.391828,0.375097,0.367582,0.375868,0.349515,...,0.552415,0.599940,0.599122,0.584046,0.550431,0.515310,0.492424,0.484102,0.514690,0
2,0.495372,0.494790,0.499717,0.515765,0.526141,0.481108,0.464553,0.459608,0.471315,0.479305,...,0.488200,0.532215,0.525498,0.515353,0.488191,0.456948,0.434431,0.429783,0.478632,0
3,0.410940,0.395382,0.426104,0.508533,0.601088,0.617408,0.639856,0.670588,0.701055,0.700051,...,0.562294,0.596344,0.573117,0.534346,0.482912,0.429992,0.386625,0.360625,0.375801,0
4,0.477980,0.476485,0.477350,0.493202,0.504986,0.465715,0.455788,0.456732,0.473373,0.484670,...,0.485730,0.536410,0.533266,0.529281,0.507363,0.480241,0.458464,0.451510,0.483707,0


In [31]:
df.to_csv('Epilepsy2_raw.csv', index=False)

In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.95, random_state=0)

In [43]:
X_train.shape

(575, 1, 178)

In [50]:
from sktime.transformations.panel.rocket import Rocket
from sktime.datasets import load_unit_test
import multiprocessing

trf = Rocket(num_kernels=20, n_jobs=multiprocessing.cpu_count()-1) 
trf.fit(X_train) 
X_train_transform = trf.transform(X_train) 
X_test_transform = trf.transform(X_test) 

In [51]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(X_train_transform, y_train)
clf.score(X_test_transform, y_test)

/home/samir/miniconda3/envs/gendis_pp/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9212814645308924

In [52]:
y_test[0:10]

array(['1', '1', '1', '1', '1', '1', '1', '0', '1', '1'], dtype='<U1')

In [60]:
truth_class_one = y_test.astype(int)
# truth_class_one[truth_class_one != 1] = 0
truth_class_one[0:10]

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1])

In [61]:
y_pred = clf.predict(X_test_transform)
y_pred[0:10]

array(['0', '1', '1', '1', '1', '1', '1', '0', '1', '1'], dtype='<U1')

In [62]:
y_pred_proba = clf.predict_proba(X_test_transform)
y_pred_proba[0:10]

array([[9.99811799e-01, 1.88201442e-04],
       [4.17768747e-03, 9.95822313e-01],
       [5.88850189e-02, 9.41114981e-01],
       [9.41364454e-03, 9.90586355e-01],
       [1.34275804e-02, 9.86572420e-01],
       [6.95200013e-03, 9.93048000e-01],
       [2.00576323e-02, 9.79942368e-01],
       [9.80038602e-01, 1.99613977e-02],
       [8.99256460e-03, 9.91007435e-01],
       [5.45982802e-03, 9.94540172e-01]])

In [63]:
proba_class_one = y_pred_proba[:, 1]
proba_class_one[0:10]

array([1.88201442e-04, 9.95822313e-01, 9.41114981e-01, 9.90586355e-01,
       9.86572420e-01, 9.93048000e-01, 9.79942368e-01, 1.99613977e-02,
       9.91007435e-01, 9.94540172e-01])

In [64]:
import numpy as np
model_error = np.absolute(truth_class_one - proba_class_one)
model_error[0:10]

array([0.9998118 , 0.00417769, 0.05888502, 0.00941364, 0.01342758,
       0.006952  , 0.02005763, 0.0199614 , 0.00899256, 0.00545983])

In [65]:
df_err = pd.DataFrame(data=X_test[:, 0, :])
df_err['truth'] = y_test
df_err['clf_pred'] = y_pred
df_err['clf_prob'] = proba_class_one
df_err['error'] = model_error
df_err.head()

,0,1,2,3,4,5,6,7,8,9,...,172,173,174,175,176,177,truth,clf_pred,clf_prob,error
0,0.442076,0.443537,0.445640,0.459069,0.463282,0.421495,0.410157,0.403137,0.411629,0.414410,...,0.560620,0.529869,0.495682,0.471787,0.462374,0.491186,1,0,0.000188,0.999812
1,0.493408,0.499578,0.505096,0.520393,0.532185,0.483627,0.462233,0.450196,0.455107,0.454522,...,0.517885,0.492915,0.461398,0.438610,0.431108,0.465812,1,1,0.995822,0.004178
2,0.472370,0.471698,0.470555,0.481921,0.485947,0.440526,0.425883,0.419869,0.431438,0.436127,...,0.545742,0.518477,0.487830,0.466040,0.459459,0.491720,1,1,0.941115,0.058885
3,0.509397,0.519009,0.524632,0.545270,0.551526,0.499860,0.477958,0.475556,0.488552,0.495912,...,0.539728,0.515977,0.491494,0.469436,0.458665,0.480502,1,1,0.990586,0.009414
4,0.521178,0.524923,0.528029,0.541799,0.547597,0.499580,0.482341,0.477908,0.488294,0.490036,...,0.559671,0.530703,0.495943,0.470742,0.468468,0.508814,1,1,0.986572,0.013428


In [66]:
df_err.to_csv('Epilepsy2_processed_error.csv', index=False)